**THINGS TO CONSIDER**

- The training essays are scored on different scales.  Perhaps the scoring system for our model should be an input, or perhaps we should have different models for different scoring systems.
- See README.md for basic guides to what's in the repository

In [11]:
import matplotlib
import matplotlib.pyplot as plt

import pandas as pd
import numpy as np

# stopwords for English
from nltk.corpus import stopwords

# English 'dictionary'
from nltk.corpus import words

# Regular expressions might be useful
import re

# Beautiful soup might be useful
from bs4 import BeautifulSoup

from sklearn.feature_extraction.text import CountVectorizer

In [16]:
train_df = pd.read_csv('data/training_set_rel3.tsv', delimiter='\t')
print train_df.head()

print '\n*****\n'

train_essays_np = train_df['essay'].values
print train_essays_np[:5]

   essay_id  essay_set                                              essay  \
0         1          1  Dear local newspaper, I think effects computer...   
1         2          1  Dear @CAPS1 @CAPS2, I believe that using compu...   
2         3          1  Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...   
3         4          1  Dear Local Newspaper, @CAPS1 I have found that...   
4         5          1  Dear @LOCATION1, I know having computers has a...   

   rater1_domain1  rater2_domain1  rater3_domain1  domain1_score  \
0               4               4             NaN              8   
1               5               4             NaN              9   
2               4               3             NaN              7   
3               5               5             NaN             10   
4               4               4             NaN              8   

   rater1_domain2  rater2_domain2  domain2_score      ...        \
0             NaN             NaN            NaN      ...    

In [17]:
test_df = pd.read_csv('data/test_set.tsv', delimiter='\t')
print test_df.head()

print '\n*****\n'

test_essays_np = test_df['essay'].values
print test_essays_np[:5]

   essay_id  essay_set                                              essay  \
0      2383          1  I believe that computers have a positive effec...   
1      2384          1  Dear @CAPS1, I know some problems have came up...   
2      2385          1  Dear to whom it @MONTH1 concern, Computers are...   
3      2386          1  Dear @CAPS1 @CAPS2, @CAPS3 has come to my atte...   
4      2387          1  Dear Local newspaper, I think that people have...   

   domain1_predictionid  domain2_predictionid  
0                  2383                   NaN  
1                  2384                   NaN  
2                  2385                   NaN  
3                  2386                   NaN  
4                  2387                   NaN  

*****

[ "I believe that computers have a positive effect on people. They help you stay in touch with family in a couple different ways they excercise your mind and hands and help you learn and make things easier. Computer's help you keep in touch 

In [18]:
valid_df = pd.read_csv('data/valid_set.tsv', delimiter='\t')
print valid_df.head()

print '\n*****\n'

valid_essays_np = valid_df['essay'].values
print valid_essays_np[:5]

   essay_id  essay_set                                              essay  \
0      1788          1  Dear @ORGANIZATION1, @CAPS1 more and more peop...   
1      1789          1  Dear @LOCATION1 Time @CAPS1 me tell you what I...   
2      1790          1  Dear Local newspaper, Have you been spending a...   
3      1791          1  Dear Readers, @CAPS1 you imagine how life woul...   
4      1792          1  Dear newspaper, I strongly believe that comput...   

   domain1_predictionid  domain2_predictionid  
0                  1788                   NaN  
1                  1789                   NaN  
2                  1790                   NaN  
3                  1791                   NaN  
4                  1792                   NaN  

*****

[ 'Dear @ORGANIZATION1, @CAPS1 more and more people start to use computers, goes more and more into the dark aged. While computer @MONTH1 be helpful in some cases, they are also making people excercize less, most @CAPS1 many people are natur